<a href="https://colab.research.google.com/github/maria-minevich/maria-minevich/blob/main/Skillbox_Chat_Bot_Day_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>



1.   Выводить текст на экран
2.   Принимать данные на вход


Чат Бот:
*    Строчка текста на вход
*    Строчка текста на выход

In [ ]:
print("Всем приветы в этом чате") # Shift+Enter

Всем приветы в этом чате


In [ ]:
text = input()
if text == "Привет":
  print("Здрасте")

if text == "Пока":
  print("Ну и до свидания")



# Обработать случаи неточного совпадения
# Две фразы? Сделать словарь для бота
# Изучить Питон

Привет
Здрасте


In [ ]:
import re  # Regular Expressions
import nltk # Natural Language Toolkit

def filter_text(text):
  text = text.lower()
  pattern = r'[^\w\s]'
  text = re.sub(pattern, "", text)
  return text

# На вход: два текста, на выход: boolean(True, False) 
# Функция isMatch вернет True, если тексты совпадают или False иначе
def is_match(text1, text2):
  # text1 = text1.lower()  # Приводим к нижнему регистру ("ПрИвет" => "привет")
  # text2 = text2.lower()

  # # Удаление знаков препинания
  # # = Удалить все кроме букв и пробелов
  # pattern = r'[^\w\s]'
  # text1 = re.sub(pattern, "", text1) # Делать замену символов в строке
  # text2 = re.sub(pattern, "", text2)
  text1 = filter_text(text1)
  text2 = filter_text(text2)

  if len(text1) == 0 or len(text2) == 0:
    return False
    
  # Проверить что одна фраза является частью другой

  # Text1 содержит text2
  if text1.find(text2) != -1:
    return True
  
  # Text2 содержит text1
  if text2.find(text1) != -1:
    return True

  # Расстояние Левенштейна (edit distance = расстояние редактирования)
  distance = nltk.edit_distance(text1, text2)  # Кол-во символов 0...Inf
  length = (len(text1) + len(text2))/2  # Средняя длина двух текстов
  score = distance / length  # 0...1

  return score < 0.6

In [ ]:
# Сколько символов нужно отредактировать "мама" = (1) => "мамы"
nltk.edit_distance("Привет братан", "Превед бротан")
# 0, 1, 2, 3...
# 0 ... 0.3 ... 0.5 ... 1

3

In [ ]:
import random

# Намерения = Intents
# Поздароваться, спросить как дела, спросить имя или чем занимаешься
# Заказать пиццу, отменить заказ, добавить больше сыра

# Конфигурация бота
BOT_CONFIG = {
    # Все намерения которые поддерживает наш бот
    "intents": {
        "hello": {
            "examples" : ["Привет", "Здарова", "Йо", "Приветос", "Хеллоу"],
            "responses": ["Здравстсвтсвтвтвуй человек", "И тебе не хворать", "Здоровее видали"],
        },
        "how_are_you": {
            "examples" : ["Как дела", "Чо каво", "Как поживаешь"],
            "responses": ["Маюсь Фигней", "Веду интенсивы", "Учу Пайтон"],
        }
    },
    # Фразы когда бот не может ответить
    "failure_phrases": ["Даже не знаю что сказать", "Поставлен в тупик", "Перефразируйте, я всего лишь бот"],
}

def printAnswer(text, examples, responses):
  for example in examples:  # Для каждого элемента списка examples
    if isMatch(text, example):  # Если пример совпадает с текстом пользователя
      print(random.choice(responses))  # Выводим на экран случайный элемент списка responses

text = input()

# Для каждого намерения, пытаемся напечатать ответ
for intent in BOT_CONFIG["intents"].values():
    printAnswer(text, intent["examples"], intent["responses"])



Приветик дорогой друг, как поживаешь вообще как дела?
И тебе не хворать
Веду интенсивы
Учу Пайтон


# День 2

Скачать: [big_bot_config.json](https://drive.google.com/file/d/1_L5CYGsO58zkB3LMBG73ezIEwYFD07Ed/view)


In [ ]:
import json

config_file = open("/content/big_bot_config.json", "r")
BIG_CONFIG = json.load(config_file)

In [ ]:
BIG_CONFIG.keys()

dict_keys(['intents', 'failure_phrases'])

In [ ]:
len(BIG_CONFIG["intents"])

439

**Задача**: Научить Модель Машинного Обучения определять intent (намерение) по тексту пользователя


строчку на вход - строчку на выход
X (входные данные) - y(выходные)
ML-модель по входным данным может предсказать выходные данные


Входные данные (X) - фразы
Выходные данные (y) - намерения


Модель обучится на наших примерах
И сможет предсказывать намерения по входной фразе


In [ ]:
X = [] # Фразы
y = [] # Намерения

# Собираем фразы и интенты из BIG_CONFIG в X,y
for name, intent in BIG_CONFIG["intents"].items():
  for example in intent["examples"]:
    X.append(example)
    y.append(name)
  for example in intent["responses"]:
    X.append(example)
    y.append(name)

len(X)

5726

In [ ]:
# Подготовка данных к обучению модели
# NLP = Natural Language Processing

# Векторайзер = превращает тексты в наборы чисел (вектора)

[sklearn](https://scikit-learn.org/stable/modules/classes.html#module-sklearn.ensemble)

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

In [ ]:
# Мама круто мыла раму => [1,2,3,4]
# Круто мама раму мыла => [2,1,4,3]
# Мыла мама круто раму => [3,1,2,4] 

In [ ]:
vectorizer = CountVectorizer() # Можно указать настройки
vectorizer.fit(X) # Учится вот эти конкретные тексты превращать в числа

CountVectorizer()

In [ ]:
arr = vectorizer.transform(["друг друг друг друг привет"]).toarray()[0]
for a in arr:
  if a!=0:
    print(a, end=',')

4,1,

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier

In [ ]:
# Классификация текста = предсказания класса(интент) по тексту(фразе)
model = LogisticRegression() # Настройки
vecX = vectorizer.transform(X) # Преобразуем тексты в вектора
model.fit(vecX, y)  # Обучаем модель
model.score(vecX, y)
# Насколько качественно обучилась модель?
# Пробуем разные модели, разные настройки

In [ ]:
model = RandomForestClassifier() # Настройки
vecX = vectorizer.transform(X) # Преобразуем тексты в вектора
model.fit(vecX, y)  # Обучаем модель
model.score(vecX, y)

0.8643031784841075

In [ ]:
model = MLPClassifier(max_iter=500, hidden_layer_sizes=(100,100,100,)) # Настройки
vecX = vectorizer.transform(X) # Преобразуем тексты в вектора
model.fit(vecX, y)  # Обучаем модель
model.score(vecX, y)

In [ ]:
import random
def get_intent_ml(text):
  vec_text = vectorizer.transform([text]) 
  intent = model.predict(vec_text)[0]
  # ToDo: выдавать ответ только если модель уверена в нем
  return intent

def get_intent(text):
  for name, intent in BIG_CONFIG["intents"].items():
    for example in intent["examples"]:
      if is_match(text, example):
        print(f"name={name} example={example}")
        return name

  return None

def bot(phrase):
  phrase = filter_text(phrase)
  # Напрямую найти ответ
  intent = get_intent(phrase)
  
  # Если напрямую интент не нашелся
  if not intent:
    # Попросить модель найти ответ
    intent = get_intent_ml(phrase)

  # Если намерение определено
  if intent:
    responses = BIG_CONFIG["intents"][intent]["responses"] # Находим варианты ответов
    return random.choice(responses)
  
  failure = BIG_CONFIG["failure_phrases"]
  return random.choice(failure)
  # Выдать Failure Phrase

# filter_text
# is_match






In [ ]:
  vec_text = vectorizer.transform(["синхрофазатрон"]) 

<1x6545 sparse matrix of type '<class 'numpy.int64'>'
	with 0 stored elements in Compressed Sparse Row format>

In [ ]:
# proba = model.predict_proba(vec_text)[0]
# intent = model.predict(vec_text)[0]
# for i in range(0, len(model.classes_)):
#   cls=model.classes_[i]
#   if cls == intent:
#     print("Intent: ", intent)
#     print("Probability: ", proba[i])

Intent:  digit_guess
Probability:  0.3155207731038464


In [ ]:
bot("как тебя зовут пацан")

In [ ]:
BIG_CONFIG["intents"]["location"]["examples"]

['Где ты находишься', 'Где ты живешь', 'Ты где']

In [ ]:
get_intent("ты где")

name=mood example=Ты как


'mood'

In [ ]:
msg = ""
exit_phrases = ["Выйти", "Выключись", "Bye", "Пока"]

while not msg in exit_phrases:
  msg = input()
  print("[BOT]: " + bot(msg))

Ты приличный?
name=bad_questions example=ты глупый
[BOT]: Я быстро учусь
Пока
name=bye example=пока
[BOT]: На связи
